#MODEL TEST

In [ ]:
import json
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

import time
import random
import numpy as np

In [ ]:
#Debemos fijar el parámetro de random seed siempre al mismo valor y después cargar el modelo para que el modelo devuelva siempre el mismo valor
# SET RANDOM SEED
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
# Cargamos los modelo y el tokenizer
#GPT2-SMALL-SPANISH
gpt2_spanish= "datificate/gpt2-small-spanish"
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained(gpt2_spanish)
tokenizer_gpt2.pad_token = tokenizer_gpt2.eos_token #Necesario porque gpt2 no tiene pad_token, por lo que asignamos eos_token


def define_labels(file):
  return max(len(item["choices"]) for item in json.load(open(file))["data"])
max_labels = define_labels("1-test.json")
print(f"El máximo número de opciones es: {max_labels}")


#cargamos el modelo con el num_labels
modelGPT2 = GPT2ForSequenceClassification.from_pretrained(gpt2_spanish, num_labels=max_labels)  #labels = opciones de respuesta
model_train = GPT2ForSequenceClassification.from_pretrained(model_name, num_labels=max_labels)
model_train.config.pad_token_id = model_train.config.eos_token_id


In [ ]:
with open("1-test.json", "r", encoding="utf-8") as f:
    test_dataset = json.load(f)["data"]

OPTION_LETTERS = ["A", "B", "C", "D", "E"]
def format_prompt(context, question, choices):
  prompt = f"Dado el siguiente contexto:\n{context}\nPregunta: {question}\nOpciones:\n" + \
         "\n".join([f"{OPTION_LETTERS[i]}. {opt}" for i, opt in enumerate(choices)]) + \
         "\nRespuesta correcta:"
  return prompt

In [ ]:
import torch.nn.functional as F
def predict_answerGPT2(inputs, choices, num_choices, temperature = 1.0):
  with torch.no_grad():
    outputs = modelGPT2(**inputs)

    logits = outputs.logits.squeeze(0)[:num_choices]  #quitamos opción extra, si la hay
    sacled_logits = logits / temperature
    probs = F.softmax(sacled_logits, dim=-1)

    predicted_index = torch.argmax(probs).item()
    if predicted_index >= num_choices:
      print(f"Error: Índice de predicción {predicted_index} fuera de rango para {num_choices} opciones. Saltamos la pregunta.")
      return None

    predicted_answer = choices[predicted_index]
  return predicted_answer


In [ ]:
correct_predictionsGPT2 = 0
total_questions = len(test_dataset)
predictionsGPT2 = []
# print(f"Total de preguntas: {total_questions}")

start_time = time.time()
for item in test_dataset:
  is_correct_prediction = False
  context = item["context"]
  question = item["question"]
  choices = [choice["text"] for choice in item["choices"]]
  correct_answer = next(choice["text"] for choice in item["choices"] if choice["type"] == "correct answer")

  num_choices = len(choices)
  if num_choices > max_labels:
    print(f"Error: Pregunta con {num_choices} opciones. Supera el máximo permitido: {max_labels}")
    continue

  prompt = format_prompt(context, question, choices)
  inputsGPT2 = tokenizer_gpt2(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)

  predicted_answerGPT2 = predict_answerGPT2(inputsGPT2, choices, num_choices)


  if correct_answer == predicted_answerGPT2:
    correct_predictionsGPT2 += 1
    is_correct_prediction = True


  predictionsGPT2.append({
    "context": context,
    "question": question,
    "choices": choices,
    "correct_answer": correct_answer,
    "predicted_answer": predicted_answerGPT2,
    "is_correct": is_correct_prediction
  })


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tiempo de ejecución: {elapsed_time:.2f} segundos")
print()

with open("predictionsGPT2.json", "w", encoding="utf-8") as f:
    json.dump(predictionsGPT2, f, indent=4, ensure_ascii=False)


print(f"Total de preguntas evaluadas: {total_questions}")
print(f"Respuestas correctas GPT2: {correct_predictionsGPT2}")
print(f"Precisión del modelo GPT2: {correct_predictionsGPT2 / total_questions:.2%}")
print()
# df = pd.DataFrame(predictionsBERT)
# print(df)